In [16]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch.utils.data import DataLoader, Subset
from typing import Dict, Any
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore')

import sys
sys.path.append('/home/guest/DemandForecasting')

from data_utils.load_data import load_and_preprocess_data
from model.baseline_models import get_baseline_model

In [17]:
dataset, scaler = load_and_preprocess_data(
    data_path='/home/guest/DemandForecasting/data/imputed_data.csv',
    time_encoded=True,
    input_len=480,
    target_len=112
)

print(f'Total dataset size: {len(dataset)}')

# split train/val/test
num_train = int(len(dataset) * 0.99)
num_val = int(len(dataset) * (1 - 0.99) / 2)

train_dataset = torch.utils.data.Subset(dataset, list(range(0, num_train)))
test_dataset = torch.utils.data.Subset(dataset, list(range(num_train + num_val, len(dataset))))
del dataset

Total dataset size: 42450000


In [18]:
print(f"Length of Dataset: {len(train_dataset)}")
x, x_dec, y = train_dataset[0]
print(f"Shapes of components of each sample: {x.shape, x_dec.shape, y.shape}")

Length of Dataset: 42025500
Shapes of components of each sample: (torch.Size([480, 11]), torch.Size([112, 10]), torch.Size([112, 1]))


In [ ]:
def compute_wape(y_true, y_pred, eps=1e-8):
    return 100 * np.sum(np.abs(y_true - y_pred)) / (np.sum(np.abs(y_true)) + eps)
def compute_daily_wape(
    y_true, 
    y_pred, 
    hours_per_day=16,
    eps=1e-8
):
    """
    y_true, y_pred: shape (N, T) or (N, T, C)
    T phải chia hết cho hours_per_day
    """
    y_true = np.array(y_true)
    y_pred = np.array(y_pred)

    # Nếu có channel dim → lấy channel cuối
    if y_true.ndim == 3:
        y_true = y_true[..., -1]
        y_pred = y_pred[..., -1]

    assert y_true.shape == y_pred.shape
    assert y_true.shape[1] % hours_per_day == 0

    num_days = y_true.shape[1] // hours_per_day

    # (N, T) → (N, num_days, hours_per_day) → sum theo giờ
    y_true_daily = y_true.reshape(-1, num_days, hours_per_day).sum(axis=2)
    y_pred_daily = y_pred.reshape(-1, num_days, hours_per_day).sum(axis=2)

    return 100 * np.sum(np.abs(y_true_daily - y_pred_daily)) / \
           (np.sum(np.abs(y_true_daily)) + eps)


def evaluate_sktime_model(
    model_fn,
    test_dataset,
    window_len=420,
    pred_len=112,
    max_samples=5000,
    hours_per_day=16
):
    all_predictions = []
    all_targets = []

    for i in tqdm(range(min(len(test_dataset), max_samples)), desc="Evaluating"):
        x, _, y = test_dataset[i]

        series = pd.Series(x[-window_len:, -1].numpy())

        model = model_fn()
        model.fit(series)

        pred = model.predict(np.arange(1, pred_len + 1))

        all_predictions.append(pred.values)
        all_targets.append(y[:, 0].numpy())

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    return {
        'WAPE': compute_daily_wape(
            all_targets,
            all_predictions,
            hours_per_day=hours_per_day
        )
    }



def evaluate_statsmodels_model(
    model_fn,
    test_dataset,
    window_len=420,
    pred_len=112,
    max_samples=5000,
    hours_per_day=16
):
    all_predictions = []
    all_targets = []

    for i in tqdm(range(min(len(test_dataset), max_samples)), desc="Evaluating"):
        x, _, y = test_dataset[i]

        series = x[-window_len:, -1].numpy()

        try:
            model = model_fn(series)
            fitted = model.fit(disp=False)
            pred = fitted.forecast(steps=pred_len)
        except Exception:
            pred = np.full(pred_len, series[-1])

        all_predictions.append(pred)
        all_targets.append(y[:, 0].numpy())

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    return {
        'WAPE': compute_daily_wape(
            all_targets,
            all_predictions,
            hours_per_day=hours_per_day
        )
    }

def evaluate_dlinear_model_dataset(
    model,
    test_dataset,
    use_decoder=True,
    device='cuda',
    max_samples=5000,
    hours_per_day=16
):
    all_predictions = []
    all_targets = []

    model.eval()
    with torch.no_grad():
        for i in tqdm(range(min(len(test_dataset), max_samples)), desc="Testing DLinear"):
            x, x_dec, y = test_dataset[i]

            x = x.unsqueeze(0).to(device)
            x_dec = x_dec.unsqueeze(0).to(device)
            y = y.unsqueeze(0).to(device)

            output = model(x, x_dec)

            if not use_decoder:
                output = output[:, :, -1:]

            all_predictions.append(output.squeeze(0).cpu().numpy())
            all_targets.append(y.squeeze(0).cpu().numpy())

    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)

    return {
        'WAPE': compute_daily_wape(
            all_targets,
            all_predictions,
            hours_per_day=hours_per_day
        )
    }


def train_and_evaluate_lstm(train_dataset, test_dataset, n_ahead=112, 
                            num_epochs=3, max_train=10000, max_test=5000, device='cuda'):
    """Train and evaluate LSTM"""
    
    # Initialize model
    model = get_baseline_model('lstm', input_size=1, hidden_size=64, 
                              num_layers=2, output_size=1, dropout=0.1)
    model = model.to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=0.001)
    criterion = nn.MSELoss()
    
    # Training
    print("Training LSTM...")
    for epoch in range(num_epochs):
        model.train()
        total_loss = 0
        
        for i in tqdm(range(min(len(train_dataset), max_train)), desc=f"Epoch {epoch+1}"):
            x, _, y = train_dataset[i]
            
            # x: (seq_len, features), extract target (last feature) only
            x_target = x[:, -1:].unsqueeze(0).to(device)  # (1, seq_len, 1)
            y = y.unsqueeze(0).to(device)  # (1, n_ahead, 1)
            
            optimizer.zero_grad()
            pred = model(x_target, n_ahead=n_ahead)
            loss = criterion(pred, y)
            loss.backward()
            optimizer.step()
            
            total_loss += loss.item()
        
        print(f"Epoch {epoch+1}, Loss: {total_loss/max_train:.4f}")
    
    # Evaluation
    print("Evaluating LSTM...")
    model.eval()
    all_predictions = []
    all_targets = []
    
    with torch.no_grad():
        for i in tqdm(range(min(len(test_dataset), max_test)), desc="Testing"):
            x, _, y = test_dataset[i]
            
            # Extract target (last feature)
            x_target = x[:, -1:].unsqueeze(0).to(device)
            pred = model(x_target, n_ahead=n_ahead)
            
            all_predictions.append(pred.cpu().numpy()[0, :, 0])
            all_targets.append(y[:, 0].numpy())
    
    all_predictions = np.array(all_predictions)
    all_targets = np.array(all_targets)
    
    return {
        'Daily_WAPE': compute_daily_wape(
            all_targets,
            all_predictions,
            hours_per_day=16
        )
    }


In [20]:
results = {}

In [21]:
results['Naive (Last)'] = evaluate_sktime_model(
    lambda: get_baseline_model('naive', strategy='last'),
    test_dataset
)

Evaluating:   0%|          | 0/5000 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 5000/5000 [00:36<00:00, 136.07it/s]


In [22]:
results['Naive (Mean)'] = evaluate_sktime_model(
    lambda: get_baseline_model('naive', strategy='mean'),
    test_dataset
)

Evaluating:   0%|          | 0/5000 [00:00<?, ?it/s]

Evaluating: 100%|██████████| 5000/5000 [00:16<00:00, 301.32it/s]


In [23]:
results['Naive (Drift)'] = evaluate_sktime_model(
    lambda: get_baseline_model('naive', strategy='drift'),
    test_dataset
)

Evaluating:   1%|          | 28/5000 [00:00<00:17, 279.96it/s]

Evaluating: 100%|██████████| 5000/5000 [00:17<00:00, 290.14it/s]


In [24]:
results['Exponential Smoothing'] = evaluate_statsmodels_model(
    get_baseline_model(
        'exponential_smoothing',
        trend='add',
        seasonal='add',
        seasonal_periods=16 # theo ngày
    ),
    test_dataset
)

Evaluating: 100%|██████████| 5000/5000 [00:01<00:00, 3010.71it/s]


In [25]:
results['ARIMA'] = evaluate_statsmodels_model(
    get_baseline_model(
        'arima',
        order=(1, 1, 1),
        seasonal_order=(1, 0, 1, 16)
    ),
    test_dataset
)

Evaluating: 100%|██████████| 5000/5000 [00:02<00:00, 2300.41it/s]


In [26]:
class Config:
    def __init__(self, data_type='imputed', use_decoder=True):
        # paths
        self.data_type = data_type
        self.data_path = f'/home/guest/DemandForecasting/data/{data_type}_data.csv'
        self.save_path = f'/home/guest/DemandForecasting/demand_forecasting/checkpoints/{data_type}_{"decoder" if use_decoder else "no_decoder"}/'
        self.log_path = f'/home/guest/DemandForecasting/demand_forecasting/logs/{data_type}_{"decoder" if use_decoder else "no_decoder"}/'
        
        # model parameters
        self.model = 'dlinear'
        self.patience = 5
        self.enable_scheduler = True
        self.seq_len = 480
        self.pred_len = 112
        self.enc_in = 11
        self.dec_in = 10
        self.use_decoder = use_decoder
        self.individual = True
        
        # training parameters
        self.batch_size = 1024
        self.lr = 0.001
        self.epochs = 20
        self.train_ratio = 0.99
        
configs = Config(data_type='imputed', use_decoder=True)

In [27]:
from model.dlinear import Model
import os

model = Model(configs)
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model.to(device)

ckpt_path = '/home/guest/DemandForecasting/demand_forecasting/checkpoints/imputed_decoder/best_dlinear_model.pth'

state_dict = torch.load(ckpt_path, map_location=device)
model.load_state_dict(state_dict)

model.eval()

Model(
  (decompsition): series_decomp(
    (moving_avg): moving_avg(
      (avg): AvgPool1d(kernel_size=(25,), stride=(1,), padding=(0,))
    )
  )
  (dec_projection): Linear(in_features=10, out_features=1, bias=True)
  (Linear_Seasonal): ModuleList(
    (0-10): 11 x Linear(in_features=480, out_features=112, bias=True)
  )
  (Linear_Trend): ModuleList(
    (0-10): 11 x Linear(in_features=480, out_features=112, bias=True)
  )
  (Linear_Decoder): ModuleList(
    (0-10): 11 x Linear(in_features=480, out_features=112, bias=True)
  )
)

In [28]:
results['DLinear (Proposed)'] = evaluate_dlinear_model_dataset(
    model,
    test_dataset,
    use_decoder=configs.use_decoder,
    device=device,
    max_samples=5000
)

Testing DLinear: 100%|██████████| 5000/5000 [00:02<00:00, 1936.11it/s]


In [29]:
results

{'Naive (Last)': {'WAPE': np.float32(93.07702)},
 'Naive (Mean)': {'WAPE': np.float32(34.818256)},
 'Naive (Drift)': {'WAPE': np.float64(106.6897770197577)},
 'Exponential Smoothing': {'WAPE': np.float32(93.07702)},
 'ARIMA': {'WAPE': np.float32(93.07702)},
 'DLinear (Proposed)': {'WAPE': np.float32(31.939238)}}